# IF3070 Foundations of Artificial Intelligence | Tugas Besar 2

Group Number: 27 TemuLawak

Group Members of TemuLawak:
- Alfandito Rais Akbar (18222037)
- Givari Alfachri (18222045)
- Muhammad Rafi Dhiyaulhaq (18222069)
- Muhammad Nurul Hakim (18222097)

## Import Libraries

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Import Dataset

In [7]:
data_train = pd.read_csv('./data/train.csv')  # Training set
data_test = pd.read_csv('./data/test.csv')  # Test set

data_train

,id,FILENAME,URL,URLLength,Domain,DomainLength,IsDomainIP,TLD,CharContinuationRate,TLDLegitimateProb,...,Pay,Crypto,HasCopyrightInfo,NoOfImage,NoOfCSS,NoOfJS,NoOfSelfRef,NoOfEmptyRef,NoOfExternalRef,label
0,1,NaN,https://www.northcm.ac.th,24.0,www.northcm.ac.th,17.0,0.0,NaN,0.800000,NaN,...,0.0,0.0,1.0,NaN,3.0,NaN,69.0,NaN,NaN,1
1,4,8135291.txt,http://uqr.to/1il1z,NaN,NaN,NaN,NaN,to,1.000000,0.000896,...,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,0
2,5,586561.txt,https://www.woolworthsrewards.com.au,35.0,www.woolworthsrewards.com.au,28.0,0.0,au,0.857143,NaN,...,1.0,0.0,1.0,33.0,7.0,8.0,15.0,NaN,2.0,1
3,6,NaN,NaN,31.0,NaN,NaN,NaN,com,0.562500,0.522907,...,1.0,0.0,1.0,24.0,5.0,14.0,NaN,NaN,NaN,1
4,11,412632.txt,NaN,NaN,www.nyprowrestling.com,22.0,0.0,NaN,1.000000,NaN,...,0.0,0.0,1.0,NaN,NaN,14.0,NaN,0.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140399,235790,49490.txt,NaN,NaN,www.slavevoyages.org,NaN,0.0,NaN,1.000000,0.079963,...,NaN,0.0,1.0,23.0,3.0,6.0,NaN,12.0,NaN,1
140400,235791,NaN,https://www.greenmountainenergy.com,34.0,www.greenmountainenergy.com,NaN,0.0,com,1.000000,0.522907,...,1.0,NaN,1.0,26.0,NaN,NaN,169.0,15.0,40.0,1
140401,235792,NaN,https://www.leadcastingcall.com,30.0,www.leadcastingcall.com,23.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,25.0,NaN,NaN,87.0,1.0,93.0,1
140402,235794,NaN,https://www.fedarb.com,21.0,NaN,14.0,NaN,NaN,1.000000,0.522907,...,0.0,0.0,1.0,NaN,36.0,NaN,102.0,NaN,NaN,1


In [9]:
is_categorical = [
    "FILENAME","URL","Domain","Title", "IsDomainIP", "TLD", "HasObfuscation", "IsHTTPS", "HasTitle",
    "HasFavicon", "Robots", "IsResponsive", "HasDescription",
    "HasExternalFormSubmit", "HasSocialNet", "HasSubmitButton",
    "HasHiddenFields", "HasPasswordField", "Bank", "Pay", "Crypto",
    "HasCopyrightInfo", "label"
]

# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [16]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...

from sklearn.model_selection import train_test_split

training_set = data_train

train_set, val_set = train_test_split(
    training_set,
    test_size=0.2,
    random_state=42
)

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer, MinMaxScaler, Normalizer, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\Users\vivobook\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\_param_validation.py)

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:
class FeatureImputer(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()
        print("Feature Imputer Initialized")

    def fit(self, data: pd.DataFrame):
        data_without_id = data.drop(columns=['id'], errors='ignore')

        self.numerical_features = data_without_id.select_dtypes(include=['number']).columns.tolist()
        self.categorical_features = data_without_id.select_dtypes(include=['object']).columns.tolist()

        self.si_numerical = SimpleImputer(strategy='median').fit(data_without_id[self.numerical_features])
        self.si_categorical = SimpleImputer(strategy='constant', fill_value='Unknown').fit(data_without_id[self.categorical_features])

        return self

    def transform(self, data: pd.DataFrame):
        data_copy = data.copy()
        data_without_id = data_copy.drop(columns=['id'], errors='ignore')

        self.numerical_features
        data_without_id[self.numerical_features] = self.si_numerical.transform(data_without_id[self.numerical_features])
        self.categorical_features
        data_without_id[self.categorical_features] = self.si_categorical.transform(data_without_id[self.categorical_features])

        if 'id' in data_copy.columns:
            data_without_id['id'] = data_copy['id']

        return data_without_id

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
class OutlierTransformator(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()
        print("Outlier Transformator Initialized")

    def fit(self, data: pd.DataFrame, y=None):
        data_without_exclusions = data.drop(columns=['id', 'label'], errors='ignore')

        numeric_columns = data_without_exclusions.select_dtypes(include=[np.number]).columns
        self.mean_values = data_without_exclusions[numeric_columns].mean(axis=0)
        return self

    def transform(self, data: pd.DataFrame):
        data_copy = data.copy()

        data_without_exclusions = data_copy.drop(columns=['id', 'label'], errors='ignore')

        numeric_columns = data_without_exclusions.select_dtypes(include=[np.number]).columns

        z_scores = (data_without_exclusions[numeric_columns] - data_without_exclusions[numeric_columns].mean(axis=0)) / data_without_exclusions[numeric_columns].std(axis=0)

        outliers = np.abs(z_scores) > 3

        for col in numeric_columns:
            data_without_exclusions.loc[outliers[col], col] = self.mean_values[col]

        if 'id' in data_copy.columns:
            data_without_exclusions['id'] = data_copy['id']
        if 'label' in data_copy.columns:
            data_without_exclusions['label'] = data_copy['label']

        return data_without_exclusions


### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
class RemoveDuplicates(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()
        print("Remove Duplicates Initialized")

    def fit(self, data: pd.DataFrame, y=None):
        duplicate_count = data.duplicated().sum()
        print(f"Number of duplicate rows: {duplicate_count}")
        return self

    def transform(self, data: pd.DataFrame):
        data_copy = data.copy()

        data_copy = data_copy.drop_duplicates()

        return data_copy

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [ ]:
class FeatureEngineering(BaseEstimator, TransformerMixin):
    def __init__(self,
                 log_features=['URLLength', 'DomainLength', 'NoOfImage', 'NoOfCSS', 'NoOfJS'],
                 interaction_features=True,
                 time_features=False):
        self.log_features = log_features
        self.interaction_features = interaction_features
        self.time_features = time_features

        self.original_columns = None

    def fit(self, X, y=None):
        self.original_columns = X.columns.tolist()
        return self

    def transform(self, X):
        cols_to_ignore = ['id', 'FILENAME', 'URL', 'Domain', 'label']
        preserved_cols = {col: X[col] for col in cols_to_ignore if col in X.columns}

        X_processed = X.drop(columns=cols_to_ignore, errors='ignore').copy()

        for feature in self.log_features:
            if feature in X_processed.columns:
                X_processed[f'{feature}_log'] = np.log1p(X_processed[feature])

        if self.interaction_features:
            if all(col in X_processed.columns for col in ['URLLength', 'DomainLength']):
                X_processed['LengthRatio'] = X_processed['URLLength'] / (X_processed['DomainLength'] + 1e-5)
                X_processed['LengthDiff'] = X_processed['URLLength'] - X_processed['DomainLength']

            if all(col in X_processed.columns for col in ['NoOfImage', 'NoOfCSS']):
                X_processed['ImageCSSRatio'] = X_processed['NoOfImage'] / (X_processed['NoOfCSS'] + 1e-5)
                X_processed['ImageCSSDiff'] = X_processed['NoOfImage'] - X_processed['NoOfCSS']

            if all(col in X_processed.columns for col in ['NoOfSelfRef', 'NoOfEmptyRef']):
                X_processed['SelfEmptyRatio'] = X_processed['NoOfSelfRef'] / (X_processed['NoOfEmptyRef'] + 1e-5)

            if all(col in X_processed.columns for col in ['NoOfImage', 'NoOfJS']):
                X_processed['ImageJSRatio'] = X_processed['NoOfImage'] / (X_processed['NoOfJS'] + 1e-5)
                X_processed['ImageJSDiff'] = X_processed['NoOfImage'] - X_processed['NoOfJS']

        for col, values in preserved_cols.items():
            X_processed[col] = values

        return X_processed

## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [ ]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [ ]:
class Scaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()

    def fit(self, X: pd.DataFrame, y=None):
        self.scaler = MinMaxScaler().fit(X.select_dtypes(include=[np.number]))
        return self

    def transform(self, X: pd.DataFrame):
        X_numeric = X.select_dtypes(include=[np.number])
        X_scaled = self.scaler.transform(X_numeric)
        X_scaled_df = pd.DataFrame(X_scaled, columns=X_numeric.columns, index=X.index)
        return pd.concat([X_scaled_df, X.drop(columns=X_numeric.columns)], axis=1)

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
class FeatureEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        super().__init__()

    def fit(self, X: pd.DataFrame, y=None):
        self.categorical_features = X.select_dtypes(include=['object']).columns
        self.encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore').fit(X[self.categorical_features])
        return self

    def transform(self, X: pd.DataFrame):
        encoded_data = self.encoder.transform(X[self.categorical_features])
        print("Shape of encoded_data:", encoded_data.shape)  # Check shape

        encoded_df = pd.DataFrame(
            encoded_data,
            columns=self.encoder.get_feature_names_out(self.categorical_features),
            index=X.index
        )

        print("Column names from get_feature_names_out:", len(self.encoder.get_feature_names_out(self.categorical_features)))  # Check column names length

        print("Encoded_df shape:", encoded_df.shape)  # Check shape

        return pd.concat([X.drop(columns=self.categorical_features), encoded_df], axis=1)

### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [ ]:
class HandleImbalance(BaseEstimator, TransformerMixin):
    def __init__(self, strategy='auto', random_state=42):
        super().__init__()
        self.smote = SMOTE(sampling_strategy=strategy, random_state=random_state)

    def fit(self, X, y):
        self.X, self.y = X, y
        return self

    def transform(self, X=None, y=None):
        if X is None or y is None:
            X, y = self.X, self.y
        X_resampled, y_resampled = self.smote.fit_resample(X, y)
        return pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled)

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

In [ ]:
pipe = Pipeline([
    ("imputer", FeatureImputer()),
    ("outlier", OutlierTransformator()),
    ("duplicate", RemoveDuplicates()),
    ("feature", FeatureEngineering()),
    ("scaler", Scaler()),
    ("encoder", FeatureEncoder()),
    ("hundlerimbalance", HandleImbalance()),
])

Feature Imputer Initialized
Outlier Transformator Initialized
Remove Duplicates Initialized


In [ ]:
train_set_clean = train_set.drop(columns=['label'])
val_set_clean = val_set.drop(columns=['label'])

train_set_transform = pipe.fit_transform(train_set_clean)
val_set_transform = pipe.transform(val_set_clean)

train_set_transform = pd.DataFrame(train_set_transform, index=train_set_clean.index)
val_set_transform = pd.DataFrame(val_set_transform, index=val_set_clean.index)

train_set_transform['label'] = train_set['label'].values
val_set_transform['label'] = val_set['label'].values

train_set_transform = train_set_transform.dropna()
val_set_transform = val_set_transform.dropna()
train_set_transform

Number of duplicate rows: 0
Shape of encoded_data: (112323, 263951)
Column names from get_feature_names_out: 263951
Encoded_df shape: (112323, 263951)


In [ ]:
y_trainset = train_set_transform['']
X_trainset = train_set_transform.drop(columns=['label','id'])
y_valset = val_set['label']
X_valset = val_set_transform.drop(columns=['label','id'])

print(f"X_trainset shape: {X_trainset.shape}")
print(f"y_trainset shape: {y_trainset.shape}")
print(f"X_valset shape: {X_valset.shape}")
print(f"y_valset shape: {y_valset.shape}")


# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `label`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter

class KNN:
    def __init__(self, k=3, metric="euclidean"):
        self.k = k
        self.metric = metric
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        """Menyimpan data pelatihan."""
        self.X_train = X
        self.y_train = y

    def _calculate_distance(self, x1, x2):
        """Menghitung jarak berdasarkan metrik yang dipilih."""
        if self.metric == "euclidean":
            return np.sqrt(np.sum((x1 - x2) ** 2))
        elif self.metric == "manhattan":
            return np.sum(np.abs(x1 - x2))
        else:
            raise ValueError("Metrik tidak dikenali. Gunakan 'euclidean' atau 'manhattan'.")

    def _get_neighbors(self, x):
        """Mengambil k tetangga terdekat untuk sampel x."""
        distances = [(self._calculate_distance(x, x_train), y_train)
                     for x_train, y_train in zip(self.X_train, self.y_train)]
        # Urutkan berdasarkan jarak dan ambil k terdekat
        sorted_neighbors = sorted(distances, key=lambda d: d[0])[:self.k]
        return [neighbor[1] for neighbor in sorted_neighbors]

    def predict(self, X):
        """Memprediksi label untuk data X."""
        predictions = [Counter(self._get_neighbors(x)).most_common(1)[0][0] for x in X]
        return np.array(predictions)

    def evaluate(self, X, y):
        """Menghitung akurasi prediksi."""
        y_pred = self.predict(X)
        accuracy = np.sum(y_pred == y) / len(y)
        return accuracy

# Contoh penggunaan
# Buat data dummy untuk percobaan
np.random.seed(42)
X_train = np.random.rand(100, 2)  # 100 sampel, 2 fitur
y_train = np.random.randint(0, 2, size=100)  # Label biner 0 atau 1

X_test = np.random.rand(20, 2)  # 20 sampel uji
y_test = np.random.randint(0, 2, size=20)  # Label uji

# Inisialisasi dan pelatihan model KNN
knn = KNN(k=5, metric="euclidean")
knn.fit(X_train, y_train)

# Prediksi dan evaluasi
y_pred = knn.predict(X_test)
accuracy = knn.evaluate(X_test, y_test)

print("Prediksi:", y_pred)
print("Akurasi:", accuracy)


In [ ]:
# Type your code here

## B. Naive Bayes

In [ ]:
# Type your code here

## C. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [ ]:
# Type your code here

## D. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `label` column.

Note: Adjust step 1 and 2 to your implementation of the preprocessing step if you don't use pipeline API from `sklearn`.

In [ ]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`